# Housing Tenure Breakdown Bar Chart

Shows the distribution of owner-occupied vs renter-occupied dwellings across neighbourhoods.

## 1. Data Reference

### Source Tables

| Table | Grain | Key Columns |
|-------|-------|-------------|
| `mart_neighbourhood_housing` | neighbourhood × year | pct_owner_occupied, pct_renter_occupied, income_quintile |

### SQL Query

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load .env from project root
load_dotenv("../../.env")

engine = create_engine(os.environ["DATABASE_URL"])

query = """
SELECT
    neighbourhood_name,
    pct_owner_occupied,
    pct_renter_occupied,
    income_quintile,
    total_rental_units,
    average_dwelling_value
FROM mart_toronto.mart_neighbourhood_housing
WHERE year = (SELECT MAX(year) FROM mart_toronto.mart_neighbourhood_housing)
  AND pct_owner_occupied IS NOT NULL
ORDER BY pct_renter_occupied DESC
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} neighbourhoods with tenure data")

Loaded 158 neighbourhoods with tenure data


### Transformation Steps

1. Filter to most recent year with tenure data
2. Melt owner/renter columns for stacked bar
3. Sort by renter percentage (highest first)

In [2]:
# Prepare for stacked bar
df_stacked = df.melt(
    id_vars=["neighbourhood_name", "income_quintile"],
    value_vars=["pct_owner_occupied", "pct_renter_occupied"],
    var_name="tenure_type",
    value_name="percentage",
)

df_stacked["tenure_type"] = df_stacked["tenure_type"].map(
    {"pct_owner_occupied": "Owner", "pct_renter_occupied": "Renter"}
)

data = df_stacked.to_dict("records")

### Sample Output

In [3]:
print("Highest Renter Neighbourhoods:")
df[
    [
        "neighbourhood_name",
        "pct_renter_occupied",
        "pct_owner_occupied",
        "income_quintile",
    ]
].head(10)

Highest Renter Neighbourhoods:


,neighbourhood_name,pct_renter_occupied,pct_owner_occupied,income_quintile
0,North St.James Town,88.9,11.1,1
1,Thorncliffe Park,85.9,14.0,1
2,South Parkdale,84.9,15.2,1
3,North Toronto,81.4,18.6,1
4,Church-Wellesley,75.7,24.3,1
5,Regent Park,70.5,29.5,1
6,Bay-Cloverhill,69.7,30.3,1
7,South Eglinton-Davisville,69.5,30.4,2
8,Oakridge,69.2,30.7,1
9,Broadview North,69.1,30.9,1


## 2. Data Visualization

### Figure Factory

Uses `create_stacked_bar` from `portfolio_app.figures.toronto.bar_charts`.

**Key Parameters:**
- `x_column`: 'neighbourhood_name'
- `value_column`: 'percentage'
- `category_column`: 'tenure_type'
- `show_percentages`: True

In [4]:
import sys

sys.path.insert(0, "../..")

from portfolio_app.figures.toronto.bar_charts import create_stacked_bar

# Show top 20 by renter percentage
top_20_names = df.head(20)["neighbourhood_name"].tolist()
data_filtered = [d for d in data if d["neighbourhood_name"] in top_20_names]

fig = create_stacked_bar(
    data=data_filtered,
    x_column="neighbourhood_name",
    value_column="percentage",
    category_column="tenure_type",
    title="Housing Tenure Mix - Top 20 Renter Neighbourhoods",
    color_map={"Owner": "#4CAF50", "Renter": "#2196F3"},
    show_percentages=True,
)

fig.show()

### City-Wide Distribution

In [5]:
# City-wide averages
print(f"City Average Owner-Occupied: {df['pct_owner_occupied'].mean():.1f}%")
print(f"City Average Renter-Occupied: {df['pct_renter_occupied'].mean():.1f}%")

# By income quintile
print("\nTenure by Income Quintile:")
df.groupby("income_quintile")[
    ["pct_owner_occupied", "pct_renter_occupied"]
].mean().round(1)

City Average Owner-Occupied: 54.0%
City Average Renter-Occupied: 46.0%

Tenure by Income Quintile:


,pct_owner_occupied,pct_renter_occupied
income_quintile,,
1,36.6,63.4
2,52.9,47.0
3,55.0,45.0
4,56.0,44.0
5,70.1,29.9
